In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pickle
import re

# Step 1: Create a dummy SQL injection dataset
def generate_sql_injection_dataset(num_samples=1000):
    np.random.seed(42)
    
    # List of SQL keywords and risky patterns
    sql_keywords = ["SELECT", "INSERT", "UPDATE", "DELETE", "DROP", "UNION", "EXEC", "OR", "AND"]
    special_characters = ["'", '"', "--", ";", "#"]
    risky_patterns = ["OR 1=1", "--", "/*", "*/", "xp_"]
    
    # Generate random strings with a mix of benign and malicious content
    def generate_random_string():
        length = np.random.randint(5, 100)
        chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789" + "".join(special_characters)
        return "".join(np.random.choice(list(chars), length))
    
    def is_malicious(string):
        return any(keyword in string.upper() for keyword in sql_keywords) or any(pattern in string for pattern in risky_patterns)
    
    data = []
    labels = []
    
    for _ in range(num_samples):
        string = generate_random_string()
        label = 1 if is_malicious(string) else 0
        
        features = [
            len(string),
            sum(c.isdigit() for c in string),
            sum(c.isalpha() for c in string),
            sum(c in special_characters for c in string),
            sum(c in special_characters for c in string) / len(string),
            int(any(keyword in string.upper() for keyword in sql_keywords)),
            int(any(pattern in string for pattern in risky_patterns))
        ]
        
        data.append(features)
        labels.append(label)
    
    return np.array(data), np.array(labels)

X, y = generate_sql_injection_dataset()

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Initialize the models
log_reg = LogisticRegression()
rf_clf = RandomForestClassifier()
dt_clf = DecisionTreeClassifier()

# Step 4: Create a voting classifier
voting_clf = VotingClassifier(
    estimators=[
        ('lr', log_reg),
        ('rf', rf_clf),
        ('dt', dt_clf)
    ],
    voting='hard'
)

# Step 5: Train the voting classifier
voting_clf.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = voting_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Step 7: Save the model as a .pkl file
model_filename = 'sql_injection_voting_classifier.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(voting_clf, file)

print(f"Model saved as {model_filename}")


Accuracy: 1.00
Model saved as sql_injection_voting_classifier.pkl


C:\Users\User 34\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
